In [1]:
import pandas as pd
import sqlite3

# Importing smaller movie_metedata via csv
# movies = pd.read_csv("movie_metadata.csv")
# movies.head()

# read movie data table from db into dataframe
con = sqlite3.connect("db/movies.db")

sql = f"""
   SELECT * FROM movie_data
   """

movies = pd.read_sql(sql, con)
# set max display to see all available columns
pd.set_option('display.max_columns', 500)
movies.head()
# Checking for duplicates...
movies['imdb_num_code'].value_counts()

tt1403981    1
tt0368447    1
tt0106308    1
tt0108399    1
tt0234215    1
tt0416449    1
tt0098084    1
tt1616195    1
tt0466893    1
tt1321860    1
tt0440963    1
tt0382765    1
tt3231054    1
tt0120390    1
tt0200530    1
tt1860353    1
tt0920458    1
tt0450278    1
tt1171222    1
tt1017451    1
tt1049405    1
tt1935179    1
tt0246894    1
tt0462538    1
tt0418763    1
tt0112346    1
tt0824747    1
tt0218378    1
tt0290095    1
tt1032755    1
            ..
tt0473444    1
tt0057076    1
tt0191636    1
tt0892769    1
tt0327597    1
tt0415978    1
tt0421994    1
tt0094721    1
tt0918927    1
tt0102070    1
tt0295254    1
tt0119468    1
tt0467200    1
tt0499549    1
tt1262416    1
tt0120102    1
tt1078588    1
tt0436613    1
tt0090329    1
tt0019729    1
tt0118548    1
tt0308055    1
tt1226271    1
tt0137523    1
tt1594562    1
tt0419749    1
tt0448011    1
tt1190617    1
tt0080661    1
tt0220099    1
Name: imdb_num_code, Length: 3637, dtype: int64

In [ ]:
# Print list of columns in smaller movies dataframe, in alphabetical order
for col in movie_data.columns.sort_values().values:
    print (col)

In [ ]:
movies_large = pd.read_csv('movies_metadata.csv')
movies_large = movies_large.set_index('imdb_id')
movies_large.head()

In [ ]:
for col in movies_large.columns.sort_values().values:
    print (col)

In [ ]:
movies.describe()
# Originally was ~4,900-5,100 records before re-pulling the data on 6/23 @ 8:21 pm...
# Believe it is because Seth dropped all N/A values
# New record count is not 3,738

In [ ]:
movies_large.describe()

In [ ]:
# Converting popularity from type 'object' to type 'numeric' (float)
movies['popularity'] = pd.to_numeric(movies['popularity'])
# Checking counts and types...nothing has changed.  
# Actually, there appears to be two more records now.  Why?
movies.describe()

In [ ]:
# Create new dataframe with just the production companies and genres from large dataset
movies_large_pc = movies_large[["production_companies","genres"]]
movies_large_pc.head()

In [ ]:
# Counting the number of production companies and genres per movie
movies_large_pc['production_companies_count'] = movies_large_pc['production_companies'].str.count('id')
movies_large_pc['genres_count'] = movies_large_pc['genres'].str.count('name')
movies_large_pc.head()

In [ ]:
# Add new columns to the existing movies table in sqllite db

# Columns straight from large dataset
movies = movies.merge(movies_large_pc[['production_companies_count','genres_count']], left_on="imdb_num_code", right_index=True, how="inner")
# Calculated columns based on occurrences of strings in certain columns
movies = movies.merge(movies_large[['popularity','vote_count','vote_average']], left_on="imdb_num_code", right_index=True, how="inner")
movies

# Definitions of new fields:

# Popularity: (Undetermined at this point on how this is calculated) - appears to be 0 to 100 rating scale.
# Vote Count: Number of users that voted for the movie on the TMDB website.
# Vote Average: Average score from the aforementioned votes.  
# Production Companies Count: Number of production companies that were a part of developing the movie.
# Genres Count: Number of genre classifications assigned to each movie.

# Website for large dataset:
# https://www.kaggle.com/rounakbanik/the-movies-dataset#movies_metadata.csv



In [ ]:
# Code to drop dups if the above cell is written multiple times
# movies = movies.drop(['popularity_x','vote_count_x','vote_average_x','production_companies_count_x',
#                     'genres_count_x','production_companies_count_y','genres_count_y','popularity_y',
#                     'vote_count_y','vote_average_y'],axis=1)


In [ ]:
# Seem to be duplicate records now...
movies['imdb_num_code'].value_counts()

# Also, why is there 128 duplicate records for tt0157472?

In [ ]:
# Remove duplicates
# removeDups2_movies = movies.drop_duplicates(subset ="imdb_num_code", 
#                      keep = 'first', inplace = True) 

In [ ]:
movies

##########################################

# Modelling

##########################################

## AdaBoost

In [2]:
import pandas as pd
import sqlite3

In [3]:
con = sqlite3.connect("db/movies.db")

sql = f"""
   SELECT * FROM movie_data
   """

movies = pd.read_sql(sql, con)
# set max display to see all available columns
pd.set_option('display.max_columns', 500)
movies.sort_index(1).head()

,actor_1_facebook_likes,actor_1_name,actor_1_score,actor_2_facebook_likes,actor_2_name,actor_2_score,actor_3_facebook_likes,actor_3_name,actor_3_score,aspect_ratio,budget,cast_total_facebook_likes,color,content_rating,country,director_facebook_likes,director_name,director_score,duration,facenumber_in_poster,genres,genres_count,gross,gross_margin,holiday_month,imdb_num_code,imdb_score,language,movie_facebook_likes,movie_imdb_link,movie_title,num_critic_for_reviews,num_user_for_reviews,num_voted_users,plot_keywords,popularity,production_companies_count,rating_numeric,release_date,release_month,revenue,success_bins,successful,title_year,tot_noms,tot_wins,total_actor_starpower,vote_average,vote_count
0,1000.0,CCH Pounder,1.520767,936.0,Joel David Moore,1.747604,855.0,Wes Studi,2.066294,1.78,237000000.0,4834,Color,PG-13,USA,0.0,James Cameron,3.091853,178.0,0.0,Action|Adventure|Fantasy|Sci-Fi,4,760505847.0,0.688365,1,tt0499549,7.9,English,33000,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,Avatar,723.0,3054.0,886204,avatar|future|marine|native|paraplegic,185.070892,4.0,3,2009-12-10,12,523505847.0,extreme success,1,2009.0,0,0,5.334665,7.2,12114.0
1,3000.0,Djimon Hounsou,3.041534,1000.0,Jason Flemyng,5.825346,855.0,Wes Studi,2.066294,2.35,45000000.0,6558,Color,R,USA,208.0,Stephen Sommers,1.766773,106.0,0.0,Action|Adventure|Horror|Sci-Fi,4,11146409.0,-3.037175,0,tt0118956,6.0,English,0,http://www.imdb.com/title/tt0118956/?ref_=fn_t...,Deep Rising,106.0,273.0,26551,creature feature|hijacker|ocean|ship|shot in t...,6.922458,3.0,4,1998-01-30,01,-33853591.0,no success,0,1998.0,0,0,10.933174,6.0,155.0
2,23000.0,Christian Bale,8.364217,909.0,Michael Greyeyes,0.582535,855.0,Wes Studi,2.066294,2.35,30000000.0,27829,Color,PG-13,USA,0.0,Terrence Malick,1.325080,150.0,0.0,Biography|Drama|History|Romance,3,12712093.0,-1.359958,1,tt0402399,6.7,English,0,http://www.imdb.com/title/tt0402399/?ref_=fn_t...,The New World,222.0,671.0,70771,captain|fort|love|native american|trade,7.694502,5.0,3,2005-12-25,12,-17287907.0,no success,0,2005.0,0,0,11.013046,6.4,336.0
3,21000.0,Robert Downey Jr.,9.884984,936.0,Joel David Moore,1.747604,722.0,Kristin Davis,2.066294,2.35,50000000.0,24664,Color,PG,USA,48.0,Brian Robbins,2.650160,98.0,0.0,Comedy|Family|Fantasy,2,61112916.0,0.181842,0,tt0393735,4.4,English,747,http://www.imdb.com/title/tt0393735/?ref_=fn_t...,The Shaggy Dog,76.0,83.0,14888,animal as human|dog movie|elevator|human becom...,4.878907,3.0,2,2006-03-09,03,11112916.0,moderate success,1,2006.0,1,0,13.698882,4.5,138.0
4,14000.0,Josh Hutcherson,2.661342,12000.0,Dwayne Johnson,1.747604,722.0,Kristin Davis,2.066294,1.85,79000000.0,27842,Color,PG,USA,62.0,Brad Peyton,1.325080,94.0,3.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,3,103812241.0,0.239011,0,tt1397514,5.8,English,0,http://www.imdb.com/title/tt1397514/?ref_=fn_t...,Journey 2: The Mysterious Island,178.0,133.0,67296,gold|hurricane|island|mountain|mysterious island,9.46307,3.0,2,2012-01-19,01,24812241.0,average success,1,2012.0,0,0,6.475240,5.8,1050.0


In [4]:
# Load libraries
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

In [5]:
# Load data
data = movies[["cast_total_facebook_likes","director_facebook_likes",
               "director_score","duration","facenumber_in_poster","genres_count","gross_margin",
               "holiday_month","imdb_score","movie_facebook_likes","production_companies_count",
               "rating_numeric","tot_noms","tot_wins","total_actor_starpower","vote_average"]]
to_predict = movies [["successful"]]
X = data
y = to_predict

In [5]:
# Split dataset into training set and test set
# 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 

In [6]:
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)

# The most important parameters are base_estimator, n_estimators, and learning_rate.

# base_estimator: It is a weak learner used to train the model. It uses DecisionTreeClassifier as default weak learner for training purpose. You can also specify different machine learning algorithms.
# n_estimators: Number of weak learners to train iteratively.
# learning_rate: It contributes to the weights of weak learners. It uses 1 as a default value.

C:\Users\ryanz\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [ ]:
# Using Different Base Learners
# You can use any ML learner as base estimator if it accepts sample weight such as Decision Tree, Support Vector Classifier.

# Load libraries
from sklearn.ensemble import AdaBoostClassifier

# Import Support Vector Classifier
from sklearn.svm import SVC
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
svc=SVC(probability=True, kernel='linear')

# Create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=50, base_estimator=svc,learning_rate=1)

# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

C:\Users\ryanz\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Naive Bayes

In [ ]:
# https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn

## Gaussian Process 